In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels as sm
import math
from scipy.optimize import linprog
from scipy.stats import bernoulli
from scipy.stats import genextreme
import progressbar
from statsmodels.api import add_constant
import time
import multiprocessing as mp

In [2]:
import warnings
warnings.filterwarnings('ignore')
widgets = ["doing task: ", progressbar.Percentage()," ",progressbar.Bar(), " ", progressbar.ETA()]

Structure:
1. Coefficient Estimation: Determine Tau_u, Grids
2. Inference: Resampling for variance and mean/ Inference, second-stage inference/  
3. Simulation
4. Real-life analysis

Estimation

In [3]:
#Linear Programming to solve coefficients
def H(x):
    return -np.log(1-x)

def generating_grids(y):
    n = len(y)
    return np.arange(1/n,1-1/n,min(0.01,1/(2*n**.7)))

def quantreg_fit(X,y,delta,grids):
    p=X.shape[0]
    n=X.shape[1]
    if n!=len(y):
        print("length not matched!")
        return
    Z = np.log(y)
    result = []
    #Calculating coefs at different grids
    #Initialize the recursive estimation
    alpha = H(grids[0])*(y>= 0)
    In = np.diag(np.repeat(1,n))
    On_vector = np.repeat(0,n)
    Op_vector = np.repeat(0,p) 
    var = np.array([]) 
    A = np.concatenate((X,-X,In,-In),axis = 0).transpose()
    #Recursive estimation
    for i in range(len(grids)-1):
        C = np.concatenate((Op_vector,Op_vector,alpha,(delta-alpha)),axis = 0)
        b = Z
        l = np.repeat(0,2*n+2*p)
        res = linprog(C, A_eq=A, b_eq=b, A_ub = -np.diag(np.repeat(1,2*n+2*p)), b_ub = l)
        beta = res.x[:p]-res.x[p:2*p]
        alpha = alpha + (H(grids[i+1])-H(grids[i]))*(y >= np.exp(X.transpose()@beta))
        result.append(beta)
    result = pd.DataFrame(result)
    result["grids"] = grids[:-1]
    result = result.set_index("grids")
    colist = []
    for i in range(len(result.columns.tolist())):
        colist.append("beta %d"%(result.columns.tolist()[i]))
    result.columns = colist
    return result

def params_inference(X,y,delta,grids,B):
    #Experiments with resamping variance and confidence interval
    p=X.shape[0]
    n=X.shape[1]
    Z = np.log(y)
    #Calculating coefs at different grids
    #Initialize the recursive estimation
    alpha = H(grids[0])*(y >= 0)
    new_alpha = [alpha]*B
    In = np.diag(np.repeat(1,n))
    On_vector = np.repeat(0,n)
    Op_vector = np.repeat(0,p) 
    A = np.concatenate((X,-X,In,-In),axis = 0).transpose()
    b = Z
    l = np.repeat(0,2*n+2*p)
    var = [] 
    l_quantile = []
    u_quantile = []
    #Recursive estimation
    random_r = np.random.exponential(size = n*B).reshape(B,n)
    for i in range(len(grids)-1):
        beta_stars = []#temporarily storing B numbers of beta_stars at each grid
        for j in range(B):
            r = random_r[j]
            C = np.concatenate((Op_vector,Op_vector,new_alpha[j]*r,(delta-new_alpha[j])*r),axis = 0)
            res = linprog(C, A_eq=A, b_eq=b, A_ub = -np.diag(np.repeat(1,2*n+2*p)), b_ub = l)
            s_beta = res.x[:p]-res.x[p:2*p]
            #Update beta and alpha
            beta_stars.append(s_beta)
            new_alpha[j] =  new_alpha[j] + (H(grids[i+1])-H(grids[i]))*(y >= np.exp(X.transpose()@s_beta))

        beta_stars = pd.DataFrame(beta_stars)
        var.append(beta_stars.var())
        l_quantile.append(beta_stars.quantile(0.025))
        u_quantile.append(beta_stars.quantile(0.975))
    
    collist = []
    for i in range(p):
        collist.append("beta %d"%(i))
    
    var = pd.DataFrame(var)
    l_quantile = pd.DataFrame(l_quantile).reset_index(drop = True)
    u_quantile = pd.DataFrame(u_quantile).reset_index(drop = True)

    for df in [var, l_quantile,u_quantile]:
        df.columns = collist
        df["grids"] = grids[:-1]

    params_result = pd.concat([var,l_quantile,u_quantile],keys=['var', 'lower', 'upper']).reset_index(level=1,drop = True)

    return params_result

def argdic_quantreg_fit(argdics):
    return quantreg_fit(argdics["X"],argdics["Y"],argdics["delta"],argdics["grids"])

def argdic_params_inference(argdics):
    return params_inference(argdics["X"],argdics["Y"],argdics["delta"],argdics["grids"],argdics["B"])

In [64]:
def T1_hypothesis_testing(X,y,delta,grids,B,l_tau,u_tau,r_v, r_0,weight_func):
    betas_hat = quantreg_fit(X,y,delta,grids)
    #Get T1
    T1 = 0
    p=X.shape[0]
    n=X.shape[1]
    Z = np.log(y)
    T1_stars = [0] * B
    #Calculating coefs at different grids
    #Initialize the recursive estimation
    alpha = H(grids[0])*(y >= 0)
    new_alpha = [alpha]*B
    In = np.diag(np.repeat(1,n))
    On_vector = np.repeat(0,n)
    Op_vector = np.repeat(0,p) 
    A = np.concatenate((X,-X,In,-In),axis = 0).transpose()
    b = Z
    l = np.repeat(0,2*n+2*p)

    #Recursive estimation
    random_r = np.random.exponential(size = n*B).reshape(B,n)
    for i in range(len(grids)-1):
        grid = grids[i]
        new_r_v = r_v(grid)
        beta_hat = betas_hat.loc[grid]
        if (grid >= l_tau) and (grid <= u_tau):
            T1 = T1 + (beta_hat@new_r_v - r_0(grid))*weight_func(grid)*0.01
        for j in range(B):
            r = random_r[j]
            C = np.concatenate((Op_vector,Op_vector,new_alpha[j]*r,(delta-new_alpha[j])*r),axis = 0)
            res = linprog(C, A_eq=A, b_eq=b, A_ub = -np.diag(np.repeat(1,2*n+2*p)), b_ub = l)
            s_beta = res.x[:p]-res.x[p:2*p]
            #Update beta and alpha
            new_alpha[j] =  new_alpha[j] + (H(grids[i+1])-H(grids[i]))*(y >= np.exp(X.transpose()@s_beta))
            # updating T1_stars
            if (grid >= l_tau) and (grid <= u_tau):
                T1_stars[j] = T1_stars[j] + (s_beta@new_r_v- beta_hat @new_r_v)*weight_func(grid)*0.01
    T1 = T1 * np.sqrt(n)
    T1_stars = pd.DataFrame(T1_stars)* np.sqrt(n)
    l_quantile = T1_stars.quantile(0.025)
    u_quantile = T1_stars.quantile(0.975)
    return {"T1":T1,"lower bound":l_quantile,"upper bound": u_quantile}

def argdic_T1_hypothesis_testing(argdics):
    return T1_hypothesis_testing(argdics["X"],argdics["Y"],argdics["delta"],argdics["grids"],argdics["B"],argdics["l_tau"],argdics["u_tau"],argdics["r_v"], argdics["r_0"],argdics["weight_func"])

def T1_hypothesis_testing_ERR(argdics):
    res = argdic_T1_hypothesis_testing(argdics)
    return (res["T1"]>res["lower bound"]) *(res["T1"]<res["upper bound"]) 


def average_cov_effect(betas_hat,grids,B,l_tau,u_tau):
    return (betas_hat.loc[l_tau:u_tau].sum()*0.01)/(u_tau - l_tau)

def cov_effect_var(X,y,delta,grids,B,l_tau,u_tau):
    p=X.shape[0]
    n=X.shape[1]
    Z = np.log(y)
    rhos_star = [0] * B
    #Calculating coefs at different grids
    #Initialize the recursive estimation
    alpha = H(grids[0])*(y >= 0)
    new_alpha = [alpha]*B
    In = np.diag(np.repeat(1,n))
    On_vector = np.repeat(0,n)
    Op_vector = np.repeat(0,p) 
    A = np.concatenate((X,-X,In,-In),axis = 0).transpose()
    b = Z
    l = np.repeat(0,2*n+2*p)

    #Recursive estimation
    random_r = np.random.exponential(size = n*B).reshape(B,n)
    for i in range(len(grids)-1):
        grid = grids[i]
        new_r_v = r_v(grid)
        for j in range(B):
            r = random_r[j]
            C = np.concatenate((Op_vector,Op_vector,new_alpha[j]*r,(delta-new_alpha[j])*r),axis = 0)
            res = linprog(C, A_eq=A, b_eq=b, A_ub = -np.diag(np.repeat(1,2*n+2*p)), b_ub = l)
            s_beta = res.x[:p]-res.x[p:2*p]
            #Update beta and alpha
            new_alpha[j] =  new_alpha[j] + (H(grids[i+1])-H(grids[i]))*(y >= np.exp(X.transpose()@s_beta))
            if (grid >= l_tau) and (grid <= u_tau):
                rhos_star[j] = rhos_star[j] + s_beta * 0.01
    rhos_star = pd.DataFrame(rhos_star)/(u_tau - l_tau)
    return rhos_star.var()

def argdic_cov_effect_var(argdics):
    return cov_effect_var(argdics["X"],argdics["Y"],argdics["delta"],argdics["grids"],argdics["B"],argdics["l_tau"],argdics["u_tau"])

## timing test

In [4]:
T1 = time.time()
n=200
b1 = .5
b2 = -.5
grids = np.arange(0.01, 0.72, 0.01)
C_u = 3.8
B = 250
Z1 = np.random.random(n)
Z2 = np.array([])
C = np.array([])
delta = np.array([])
epsilon = -genextreme.rvs(1, size = n)
C_u = 5
for i in range(n):
    z = bernoulli.rvs(.5)
    Z2 = np.append(Z2,z)
    if z == 0:
        C= np.append(C, np.random.uniform(0,C_u))
    else:
        C= np.append(C, np.random.uniform(0.1,C_u))
T = np.exp(b1*Z1+b2*Z2+epsilon)
X = np.vstack((Z1,Z2,np.repeat(1,n)))
delta = (T<=C)
Y = np.minimum(T,C)
params = quantreg_fit(X,Y,delta,grids)
inference = params_inference(X,Y,delta,grids,B)
T2 = time.time()
print('程序运行时间:%s秒' % ((T2 - T1)))

程序运行时间:116.70840954780579秒


Simulation
1. AFT model
$$Log(T) = b_1Z_1+b_2Z_2+\epsilon$$
$$\epsilon ～ GEV;Z_1～unif(0,1);Z_2～Bernolli(0.5) $$
$$ if Z_2 = 0,  C～unif(0,c_u); if Z_2 = 1,  C～unif(0.1,c_u) $$

1.1 
$$c_u = 3.8, b_1 = 0.5, b_2 = -0.5;$$
1.2
$$C_u = 5, b_1 = 0, b_2 = -0.5$$

In [41]:
# Simulation Setting one
def r_v(tau):
    return (np.array([[1,0,0],[0,1,0]]).transpose())

def r_0(tau):
    return np.array([0,0])

def weight_func(tau):
    return 1

l_tau = grids[0]
u_tau = grids[-2]

def setone_simulate(n=200,b1=.5,b2=-.5,grids=np.arange(0.01, 0.72, 0.01),C_u = 5, B = 250, l_tau=l_tau,u_tau = u_tau,r_v=r_v, r_0=r_0,weight_func =weight_func ,samplepath = 1000):
    args = []
    for i in range(samplepath):
        Z1 = np.random.random(n)
        Z2 = np.array([])
        C = np.array([])
        delta = np.array([])
        epsilon = genextreme.rvs(1, size = n)
        for j in range(n):
            z = bernoulli.rvs(.5)
            Z2 = np.append(Z2,z)
            if z == 0:
                C= np.append(C, np.random.uniform(0,C_u))
            else:
                C= np.append(C, np.random.uniform(0.1,C_u))
        T = np.exp(b1*Z1+b2*Z2+epsilon)
        X = np.vstack((Z1,Z2,np.repeat(1,n)))
        delta = (T<=C)
        Y = np.minimum(T,C)
        args.append({"X":X,"Y":Y,"grids":grids,"delta":delta,"B":B,"l_tau":l_tau,"u_tau":u_tau,"r_v":r_v, "r_0":r_0,"weight_func":weight_func})
    return args

grids = np.arange(0.01, 0.72, 0.01)
target_grids = [grids[9],grids[29],grids[49],grids[69]]

def temp_quantreg(argdics):
    return argdic_quantreg_fit(argdics).loc[target_grids]

def temp_inference(argdics):
    target_grids = [grids[9],grids[29],grids[49],grids[69]]
    df = argdic_params_inference(argdics)
    return df[df["grids"].isin(target_grids)]

In [32]:
#Simulation Setting 1 Sample 1
sample_path = 500
sample1 = setone_simulate(samplepath = 500,B = 200, C_u = 5,b1 = 0)

p = 3

true_betas = []
for i in range(4):
    true_betas.append(np.array([0,-0.5,genextreme.ppf(target_grids[i], 1)]))

#Restoring simulated samples:
sample_betas = [np.repeat(0,p)]*4
sample_vars = [np.repeat(0,p)]*4
sample_CIs = [np.repeat(0,p)]*4
dfl = []
for i in range(4):
    dfl.append(pd.DataFrame(columns = ["Bias", "AveVar","EmpVar","Cov95"]))
    
if __name__ == '__main__':
    T1 = time.time()
    pool = mp.Pool(processes=40)
    params = pool.map(temp_inference,sample1)
    all_betas = pool.map(temp_quantreg,sample1) #a list of betas dfs
    T2 = time.time()
    print('程序运行时间:%s毫秒' % ((T2 - T1)*1000))

程序运行时间:8020699.6150016785毫秒


In [33]:
len(params)

500

In [37]:
p = 3
#Restoring simulated samples:
sample_betas = [np.repeat(0,p)]*4
sample_vars = [np.repeat(0,p)]*4
sample_CIs = [np.repeat(0,p)]*4
dfl = []
for i in range(4):
    dfl.append(pd.DataFrame(columns = ["Bias", "AveVar","EmpVar","Cov95"]))
    
for k in range(len(target_grids)):
    true_beta = true_betas[k]
    grid = target_grids[k]
    for j in range(sample_path):
        beta = np.array(all_betas[j].loc[grid])
        sample_betas[k] = np.vstack((sample_betas[k], beta))
        temp_var,temp_l,temp_u = params[j].loc["var"].set_index("grids"),params[j].loc["lower"].set_index("grids"),params[j].loc["upper"].set_index("grids")
        sample_vars[k] =  np.vstack((sample_vars[k], np.array(temp_var.loc[grid]))) 
        l = true_beta - np.sqrt(np.array(temp_var.loc[grid])) * 1.96
        u = true_beta + np.sqrt(np.array(temp_var.loc[grid])) * 1.96
        temp = (beta>l) * (beta<u)
        sample_CIs[k] =  np.vstack((sample_CIs[k],temp)) 

#Calculate empirical bias and variance
# empirical_mean = []
# empirical_var = []
# empirical_coverage = []
# average_var = []
for k in range(4):
    sample_betas[k] = pd.DataFrame(sample_betas[k][1:])
    sample_vars[k] = pd.DataFrame(sample_vars[k][1:])
    sample_CIs[k] = pd.DataFrame(sample_CIs[k][1:])
#     empirical_mean.append(np.array(sample_betas.mean()))
#     empirical_var.append(np.array(sample_betas.var()))
#     empirical_coverage.append(sum(sample_CIs))
#     average_var.append(np.array(sample_vars.mean()))
    dfl[k]["Bias"] = sample_betas[k].mean() - true_betas[k] 
    dfl[k]["AveVar"] = sample_vars[k].mean()
    dfl[k]["EmpVar"] = sample_betas[k].var()
    dfl[k]["Cov95"] = sample_CIs[k].sum()/sample_path
T2 = time.time()
print('程序运行时间:%s毫秒' % ((T2 - T1)*1000))
params_set1 = pd.concat(dfl,keys = ["tau = 0.1","tau = 0.3","tau = 0.5","tau = 0.7"])

程序运行时间:8174449.815273285毫秒


In [38]:
params_set1

Bias    AveVar    EmpVar  Cov95
tau = 0.1 0 -0.015590  0.635135  0.530051  0.950
          1  0.013489  0.222246  0.181487  0.956
          2 -0.001507  0.275272  0.245703  0.932
tau = 0.3 0 -0.028725  0.163289  0.149564  0.942
          1 -0.000691  0.054713  0.047616  0.962
          2  0.025672  0.068963  0.069358  0.918
tau = 0.5 0 -0.009326  0.075044  0.068849  0.956
          1 -0.006312  0.024788  0.020889  0.964
          2  0.014723  0.031450  0.031365  0.918
tau = 0.7 0 -0.004078  0.034028  0.031756  0.934
          1 -0.003606  0.011559  0.010054  0.944
          2  0.010519  0.014669  0.012923  0.940

# Hypothesis Testing

In [42]:
sample1 = setone_simulate(samplepath = 500,B = 200, C_u = 5,b1 = 0)

In [45]:
if __name__ == '__main__':
    T1 = time.time()
    print(T1)
    pool = mp.Pool(processes=40)
    T1_ERR = pool.map(T1_hypothesis_testing_ERR,sample1)
    T2 = time.time()
    print('程序运行时间:%s毫秒' % ((T2 - T1)*1000))

1691509501.240801
程序运行时间:2999538.2816791534毫秒


In [48]:
T1_ERR = pd.DataFrame(T1_ERR)/500
T1_ERR.columns = ["b1","b2"]
T1_ERR.sum()

b1    0.910
b2    0.226
dtype: float64

# Average Covariate Effect Estimation

In [67]:
if __name__ == '__main__':
    T1 = time.time()
    pool = mp.Pool(processes=40)
    ace_var = pool.map(argdic_cov_effect_var,sample1)
    all_betas = pool.map(argdic_quantreg_fit,sample1) 
    ace = []
    ace_inference = pd.DataFrame()
    for i in range(len(all_betas)):
        ace.append(average_cov_effect(all_betas[i],grids,B,l_tau,u_tau))
    ace = pd.DataFrame(ave)
    ace_var = pd.DataFrame(ace_var)
    ace_var.columns = ["beta 0", "beta 1", "beta 2"]
    ace_inference["AveEst"] = ace.mean()
    ace_inference["EmpVar"] = ace.var()
    ace_inference["AveVar"] = ace_var.mean()

In [78]:
ace_inference

,AveEst,EmpVar,AveVar
beta 0,-0.011775,0.110948,0.101144
beta 1,-0.511035,0.036787,0.035457
beta 2,-0.306335,0.044722,0.043887
